# Neo4j Graph Database ETL Pipeline - GDSC-007 Data Upload

## Σύνοψη Διαδικασίας

Αυτό το notebook υλοποιεί ένα ολοκληρωμένο **ETL (Extract, Transform, Load) pipeline** για τη φόρτωση πειραματικών δεδομένων από το Genomics of Drug Sensitivity in Cancer (GDSC) dataset στη graph database Neo4j. Η διαδικασία μετατρέπει tabular δεδομένα σε **γραφική αναπαράσταση** (graph structure) με κόμβους (nodes) και σχέσεις (relationships), επιτρέποντας πολύπλοκα queries που αξιοποιούν τις συνδέσεις μεταξύ βιολογικών οντοτήτων.

Τα dataset GDSC-00Χ περιέχουν αποτελέσματα από πειράματα συνδυαστικής χημειοθεραπείας, όπου δοκιμάζονται συνδυασμοί φαρμάκων σε διάφορες κυτταρικές σειρές καρκίνου. Τα δεδομένα περιλαμβάνουν μετρήσεις συνέργειας (Bliss independence, HSA), φαρμακοκινητικές παραμέτρους, και χαρακτηριστικά των κυτταρικών σειρών.

## Αρχιτεκτονική Σύνδεσης και Βοηθητικές Συναρτήσεις

### Neo4j Connection Management

Η συνάρτηση `connect_neo4j_prod()` διαχειρίζεται τη σύνδεση με τη **Neo4j cloud database**:
- **URI**: `<NEO4J_URI>` - Το `+s` υποδηλώνει encrypted connection (SSL/TLS)
- **Authentication**: Χρησιμοποιεί username/password για ασφαλή πρόσβαση
- **Connection pooling**: Το `max_connection_lifetime=200` ορίζει το μέγιστο χρόνο ζωής μιας σύνδεσης σε δευτερόλεπτα

Η σύνδεση περιτυλίγεται σε try-except block για graceful error handling, εμφανίζοντας διαγνωστικά μηνύματα επιτυχίας ή αποτυχίας.

### Batch Processing Strategy

Η συνάρτηση `chunk_dataframe()` υλοποιεί **batch processing** για αποδοτική φόρτωση μεγάλων datasets:
- Διαιρεί το DataFrame σε μικρότερα chunks (συνήθως 20,000 γραμμές)
- Αποφεύγει memory overflow και timeouts
- Επιτρέπει progress tracking και resume σε περίπτωση αποτυχίας
- Βελτιώνει την απόδοση του Neo4j με οργανωμένες transactions

### Cypher Query Execution

Η συνάρτηση `run_on_neo4j()` εκτελεί **Cypher queries** (η query γλώσσα του Neo4j) με:
- **Transaction management**: Χρήση explicit transactions για atomicity
- **Parameterized queries**: Αποφυγή SQL injection και βελτίωση performance μέσω query caching
- **Error handling**: Comprehensive exception catching με traceback για debugging
- **Result processing**: Επιστρέφει είτε τα αποτελέσματα είτε τα counters (nodes created, relationships created, etc.)

## Φόρτωση Δεδομένων και Header Mapping

Κάθε συνάρτηση φόρτωσης ξεκινά με την ανάγνωση του CSV αρχείου και την αντιστοίχιση των στηλών σε ουσιαστικά ονόματα. Το αρχικό CSV έχει 52 στήλες που αντιπροσωπεύουν:

**Μεταδεδομένα Πειράματος:**
- `comb_id`: Μοναδικός αναγνωριστικός κωδικός συνδυασμού
- `CELL_LINE_NAME`, `MASTER_CELL_ID`, `COSMIC_ID`: Αναγνωριστικά κυτταρικής σειράς
- `TISSUE`, `CANCER_TYPE`: Κατηγοριοποίηση καρκίνου

**Δεδομένα Φαρμάκου 1 (lib1):**
- `lib1_ID`, `lib1_name`, `lib1_tag`: Αναγνωριστικά
- `lib1_minc`, `lib1_maxc`: Εύρος συγκεντρώσεων
- `lib1_RMSE`, `lib1_MaxE`, `lib1_IC50_ln`: Φαρμακοκινητικές παράμετροι
- `lib1_target`, `lib1_pathway`: Βιολογικοί στόχοι

**Δεδομένα Φαρμάκου 2 (lib2):** Αντίστοιχα με lib1

**Μετρήσεις Συνέργειας:**
- `Bliss_*`: Bliss independence model metrics
- `HSA_*`: Highest Single Agent model metrics
- `Delta_*`: Διαφορές μεταξύ μονοθεραπείας και συνδυασμού

**Βιολογικές Μετρήσεις:**
- `day1_*`: Μετρήσεις πρώτης ημέρας (viability, intensity, inhibition)
- `growth_rate`, `doubling_time`: Χαρακτηριστικά ανάπτυξης

## Δημιουργία Node Types

### 1. Drug Nodes (`drugs()`)

Δημιουργεί κόμβους τύπου **Drug** για όλα τα φάρμακα που εμπλέκονται στα πειράματα:

```cypher
MERGE (n:Drug{id:row.lib1_ID})
```

**Ιδιότητες:**
- `id`: Μοναδικό αναγνωριστικό (primary key)
- `name`: Ονομασία φαρμάκου
- `tag`: Κατηγορία/ετικέτα
- `maxc`, `minc`: Εύρος συγκεντρώσεων που δοκιμάστηκαν

Η χρήση `MERGE` αντί για `CREATE` διασφαλίζει ότι δεν δημιουργούνται **duplicates** - αν το φάρμακο υπάρχει ήδη, απλώς ενημερώνονται οι ιδιότητές του. Δημιουργούνται ξεχωριστά nodes για lib1 και lib2, αλλά λόγω του MERGE, το ίδιο φάρμακο που εμφανίζεται σε πολλαπλούς συνδυασμούς θα αναπαρασταθεί από έναν μόνο κόμβο.

### 2. Gene Nodes (`genes()`)

Δημιουργεί κόμβους **Gene** για τους γονιδιακούς στόχους των φαρμάκων:

```cypher
MERGE (n:Gene{name:row.lib1_target})
```

Οι κόμβοι Gene είναι απλοί με μόνο ιδιότητα το `name`, αλλά είναι κρίσιμοι για τη σύνδεση με:
- Τα φάρμακα που τους στοχεύουν (TARGETS relationship)
- Τα pathways στα οποία συμμετέχουν (WITHIN relationship)
- Τις κυτταρικές σειρές όπου εκφράζονται (EXPRESSED_IN relationship)

### 3. Pathway Nodes (`pathways()`)

Δημιουργεί κόμβους **Pathway** για τα βιολογικά μονοπάτια:

```cypher
MERGE (n:Pathway{name:row.lib1_pathway})
```

Τα pathways αντιπροσωπεύουν **σηματοδοτικά μονοπάτια** ή λειτουργικές ομάδες πρωτεϊνών (π.χ. EGFR signaling, PI3K/AKT pathway, Apoptosis regulation). Η κατηγοριοποίηση αυτή είναι σημαντική για:
- Κατανόηση του μηχανισμού δράσης των φαρμάκων
- Εντοπισμό πιθανών συνεργιών (φάρμακα που στοχεύουν complementary pathways)
- Στρατηφικοποίηση αποτελεσμάτων βάσει βιολογικής λειτουργίας

### 4. CellLine Nodes (`celllines()`)

Δημιουργεί κόμβους **CellLine** για τις κυτταρικές σειρές καρκίνου:

```cypher
MERGE (n:CellLine{id:row.MASTER_CELL_ID})
SET n.cancer_type = row.CANCER_TYPE,
    n.tissue = row.TISSUE,
    n.name = row.CELL_LINE_NAME
```

**Ιδιότητες:**
- `id`: MASTER_CELL_ID (μοναδικός αναγνωριστικός)
- `name`: Όνομα κυτταρικής σειράς (π.χ. MCF7, HeLa)
- `cancer_type`: Τύπος καρκίνου (π.χ. Breast Cancer, Lung Adenocarcinoma)
- `tissue`: Ιστός προέλευσης (π.χ. Breast, Lung, Colon)

Αυτές οι ιδιότητες επιτρέπουν queries όπως "βρες όλα τα πειράματα σε lung cancer cell lines" ή "ποια φάρμακα είναι αποτελεσματικά σε breast tissue".

### 5. Combination Nodes (`combinations()`)

Δημιουργεί κόμβους **Combination** που αποτελούν το **κέντρο του γράφου** - κάθε combination αντιπροσωπεύει ένα μοναδικό πείραμα:

```cypher
MERGE (n:Combination{id:row.comb_id})
```

**Περιέχει 30+ ιδιότητες** που αποθηκεύουν όλα τα πειραματικά αποτελέσματα:

**Bliss Metrics:**
- `bliss_window`: Το κύριο metric συνέργειας (target μεταβλητή στο ML μοντέλο)
- `bliss_matrix`, `bliss_matrix_so`: Matrices συνέργειας
- `bliss_synergistic_wells`: Πλήθος πηγαδιών με συνέργεια

**HSA Metrics:** Εναλλακτική μέθοδος μέτρησης συνέργειας

**Delta Metrics:** Υπολογισμοί διαφορών για σύγκριση μονοθεραπείας vs συνδυασμού

**Biological Measurements:**
- `day1_inhibition_scale`, `day1_intensity_mean`, `day1_viability_mean`
- `growth_rate`, `doubling_time`

**Metadata:**
- `data_source`: '007' (για tracking ποια batch είναι)
- `drug1_name`, `drug2_name`: Denormalized για ευκολία στα queries

Αυτός ο κόμβος είναι **fact table** στη γραφική αναπαράσταση, ανάλογο με το fact table σε star schema.

## Δημιουργία Relationships

Οι σχέσεις ορίζουν τη **σημασιολογία του γράφου** και επιτρέπουν traversals για ανακάλυψη patterns.

### CONTAINS Relationships (`contains_rels()`)

Συνδέει **Combination → Drug**:

```cypher
MATCH (m:Combination {id:row.comb_id})
MATCH (r:Drug{id:row.lib1_ID})
MERGE (m)-[f:CONTAINS]->(r)
```

Κάθε Combination node συνδέεται με **δύο** Drug nodes (lib1 και lib2). Αυτό επιτρέπει queries όπως:
- "Ποιοι συνδυασμοί περιέχουν το φάρμακο X;"
- "Με ποια άλλα φάρμακα έχει συνδυαστεί το φάρμακο Y;"

### TESTED_ON Relationships (`tested_rels()`)

Συνδέει **Combination → CellLine**, **Drug → CellLine**:

```cypher
MERGE (m)-[f:TESTED_ON]->(r)
```

Δημιουργούνται **τρεις** τύποι σχέσεων:
1. Combination tested on CellLine (το πείραμα έγινε σε ποια κυτταρική σειρά)
2. Drug1 tested on CellLine (ποια φάρμακα έχουν δοκιμαστεί σε ποιες σειρές)
3. Drug2 tested on CellLine

Αυτό επιτρέπει:
- "Ποια φάρμακα έχουν δοκιμαστεί στη MCF7 cellline;"
- "Σε ποιες celllines έχει δοκιμαστεί ο συνδυασμός X-Y;"

### TARGETS Relationships (`targets_rels()`)

Συνδέει **Drug → Pathway** και **Drug → Gene**:

```cypher
MATCH (m:Drug{id:row.lib1_ID})
MATCH (r:Pathway{name:row.lib1_pathway})
MERGE (m)-[f:TARGETS]->(r)
```

Αποθηκεύει τη **βιολογική γνώση** για το μηχανισμό δράσης των φαρμάκων:
- "Ποια φάρμακα στοχεύουν το EGFR pathway;"
- "Ποιο γονίδιο στοχεύει το φάρμακο Olaparib;" (PARP1)

Δημιουργούνται **τέσσερις** relationships:
1. Drug1 → Pathway1
2. Drug2 → Pathway2  
3. Drug1 → Gene1
4. Drug2 → Gene2

### WITHIN Relationships (`within_rels()`)

Συνδέει **Gene → Pathway**:

```cypher
MATCH (m:Gene{name:row.lib1_target})
MATCH (r:Pathway{name:row.lib1_pathway})
MERGE (m)-[f:WITHIN]->(r)
```

Ορίζει τη **biological hierarchy**: ποια γονίδια ανήκουν σε ποια pathways. Αυτό επιτρέπει multi-hop queries όπως:
- "Βρες φάρμακα που στοχεύουν γονίδια στο PI3K pathway"
- "Ποια γονίδια του apoptosis pathway στοχεύονται από τα φάρμακά μας;"

### EXPRESSED_IN Relationships (`expressed_rels()`)

Συνδέει **Gene → CellLine**:

```cypher
MATCH (m:Gene{name:row.lib1_target})
MATCH (r:CellLine{id:row.MASTER_CELL_ID})
MERGE (m)-[f:EXPRESSED_IN]->(r)
```

Καταγράφει ότι ένα **gene target εκφράζεται σε μια κυτταρική σειρά**. Αυτό είναι σημαντικό επειδή:
- Αν ένα γονίδιο δεν εκφράζεται, το φάρμακο που το στοχεύει μπορεί να μην είναι αποτελεσματικό
- Επιτρέπει stratification: "Ποια φάρμακα είναι καλύτερα σε celllines που εκφράζουν BRCA1;"

## Εκτέλεση με APOC Periodic Iterate

Όλα τα queries χρησιμοποιούν την **APOC** (Awesome Procedures On Cypher) βιβλιοθήκη, συγκεκριμένα το `apoc.periodic.iterate`:

```cypher
CALL apoc.periodic.iterate(
    "UNWIND $parameters as row RETURN row",  // Outer query: επιστρέφει γραμμές
    "MERGE (n:Drug{id:row.lib1_ID}) ...",    // Inner query: επεξεργάζεται κάθε γραμμή
    {batchSize:20000, parallel: false, params:{parameters: $parameters}}
)
```

**Πλεονεκτήματα:**
- **Batch processing**: Χωρίζει αυτόματα σε batches για αποφυγή memory issues
- **Progress tracking**: Επιστρέφει counters για monitoring
- **Transaction management**: Κάθε batch είναι ξεχωριστό transaction
- **Resilience**: Αν αποτύχει ένα batch, τα υπόλοιπα συνεχίζουν

Η παράμετρος `parallel: false` διασφαλίζει **σειριακή εκτέλεση** για αποφυγή deadlocks όταν δημιουργούνται relationships που μπορεί να ανταγωνίζονται για τους ίδιους κόμβους.

## Orchestration με Main Function

Η συνάρτηση `main()` ορχηστρώνει τη **σειρά εκτέλεσης**:

```python
def main():
    drugs()        # 1. Πρώτα δημιούργησε τους Drug nodes
    genes()        # 2. Μετά τους Gene nodes
    pathways()     # 3. Μετά τους Pathway nodes
    celllines()    # 4. Μετά τους CellLine nodes
    combinations() # 5. Μετά τους Combination nodes
    contains_rels()   # 6. Τώρα μπορούμε να φτιάξουμε CONTAINS relationships
    tested_rels()     # 7. TESTED_ON relationships
    targets_rels()    # 8. TARGETS relationships
    within_rels()     # 9. WITHIN relationships
    expressed_rels()  # 10. EXPRESSED_IN relationships
```

Η **σειρά είναι κρίσιμη**: πρέπει να υπάρχουν πρώτα οι κόμβοι πριν δημιουργηθούν οι σχέσεις που τους συνδέουν. Τα σχόλια στον κώδικα (με #) επιτρέπουν την επιλεκτική εκτέλεση συγκεκριμένων βημάτων για testing ή incremental updates.

Το τελικό αποτέλεσμα είναι ένας **πλούσιος βιολογικός γράφος** όπου μπορούν να εκτελεστούν σύνθετα queries για ανακάλυψη φαρμακολογικών patterns, πρόβλεψη συνεργιών, και stratification treatments βάσει βιολογικών χαρακτηριστικών των καρκίνων.

In [ ]:
from neo4j import GraphDatabase
import os
import pandas as pd
import warnings
import traceback

In [ ]:
# Set Neo4j credentials as environment variables for security.
os.environ["NEO4J_URI"] = "..." 
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "..."

In [ ]:
#Place your file here
file_path_deployments= 'gdsc-007_matrix_results.csv' #only the name of the file - "name.xlsx", the code uses '007' file for reference, just change the file name to the one you want to use to upload the rest
deps_df= pd.read_csv(file_path_deployments)
deps_df.head(10)

,comb_id,CELL_LINE_NAME,MASTER_CELL_ID,COSMIC_ID,TISSUE,CANCER_TYPE,lib1_tag,lib1_ID,lib1_name,lib1_minc,...,HSA_window_dose1,HSA_window_dose2,HSA_window_SO,HSA_window_SO_dose1,HSA_window_SO_dose2,day1_intensity_mean,day1_viability_mean,day1_inhibition_scale,growth_rate,doubling_time
0,1,A2058,652,906792,Skin,Melanoma,L56,2435,AZD6738,0.010,...,D5,D5,0.074474,D5,D5,27426.145292,0.955255,0.044745,0.064575,1114.983622
1,2,A2058,652,906792,Skin,Melanoma,L57,2436,AZD1775,0.001,...,D5,D5,0.106677,D3,D1,27426.145292,0.955255,0.044745,0.064575,1114.983622
2,3,A2058,652,906792,Skin,Melanoma,L58,2436,AZD1775,0.001,...,D3,D1,0.323042,D3,D1,27426.145292,0.955255,0.044745,0.064575,1114.983622
3,4,A2058,652,906792,Skin,Melanoma,L59,2428,AZD2811,0.001,...,D3,D5,0.118349,D3,D5,27426.145292,0.955255,0.044745,0.064575,1114.983622
4,5,A2058,652,906792,Skin,Melanoma,L60,2428,AZD2811,0.001,...,D5,D3,0.091989,D4,D3,27426.145292,0.955255,0.044745,0.064575,1114.983622
5,6,A2058,652,906792,Skin,Melanoma,L61,2428,AZD2811,0.001,...,D1,D3,0.123039,D1,D5,27426.145292,0.955255,0.044745,0.064575,1114.983622
6,7,A2058,652,906792,Skin,Melanoma,L63,2429,AZD0156,0.001,...,D5,D5,0.070973,D1,D3,27426.145292,0.955255,0.044745,0.064575,1114.983622
7,8,A2058,652,906792,Skin,Melanoma,L65,2430,AZD7648,0.010,...,D4,D3,0.059859,D1,D1,27426.145292,0.955255,0.044745,0.064575,1114.983622
8,9,A2058,652,906792,Skin,Melanoma,L62,2429,AZD0156,0.001,...,D3,D5,0.129465,D4,D1,27426.145292,0.955255,0.044745,0.064575,1114.983622
9,10,A2058,652,906792,Skin,Melanoma,L64,2429,AZD0156,0.001,...,D3,D5,0.095430,D2,D3,27426.145292,0.955255,0.044745,0.064575,1114.983622


In [4]:
print(len(deps_df))

19536


In [ ]:
def connect_neo4j_prod():

    uri = os.getenv("NEO4J_URI")
    username = os.getenv("NEO4J_USERNAME")
    password = os.getenv("NEO4J_PASSWORD")  
    if not all([uri, username, password]):
        raise ValueError("Missing Neo4j credentials. Set NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD.")

    graph = None
    try:
        graph = GraphDatabase.driver(uri, auth=(username, password),max_connection_lifetime=200)
        print(
            "##################         Connected to NEO4j KG         #####################"
        )
    except:
        print("##################Could not Connect to NEO4j#####################")
        traceback.print_exc()
    return graph

def chunk_dataframe(dataframe, chunk_size):
    """
    Returns a list of dataframes consiting of at maxium the specified chunk size
   :param dataframe: Input dataframe
   :param chunk_size: Desired chunk size to split dataframe
    """
    return [dataframe[x: x+chunk_size] for x in range(0, len(dataframe), chunk_size)]

def run_on_neo4j(cypher, p , session_por):
    try:
            tx = session_por.begin_transaction()
            curs = tx.run(cypher, p)
            result = [record.data() for record in curs]
            if len(list(result)) == 0:
                result = curs.consume().counters
            tx.commit()
    except Exception as exception_message:
        print(f"Error during Cypher query execution: {exception_message}")
        traceback.print_exc() 
        result = 'error'

    return result



In [ ]:
def drugs():

    headers_mat=["comb_id","CELL_LINE_NAME","MASTER_CELL_ID","COSMIC_ID","TISSUE", "CANCER_TYPE","lib1_tag","lib1_ID","lib1_name","lib1_minc",'lib1_maxc','lib1_RMSE','lib1_MaxE',
                 'lib1_IC50_ln','lib1_target','lib1_pathway','lib2_tag','lib2_ID','lib2_name','lib2_minc','lib2_maxc','lib2_RMSE','lib2_MaxE','lib2_IC50_ln','lib2_target',
                 'lib2_pathway','combo_MaxE','Delta_MaxE_lib1','Delta_MaxE_lib2','Delta_combo_MaxE_day1','Bliss_synergistic_wells','Bliss_matrix','Bliss_matrix_SO',
                 'window_size','Bliss_window','Bliss_window_dose1','Bliss_window_dose2','Bliss_window_SO','Bliss_window_SO_dose1','Bliss_window_SO_dose2',
                 'HSA_synergistic_wells','HSA_matrix','HSA_matrix_SO','HSA_window','HSA_window_dose1','HSA_window_dose2','HSA_window_SO','HSA_window_SO_dose1',
                 'HSA_window_SO_dose2','day1_intensity_mean','day1_viability_mean','day1_inhibition_scale','growth_rate','doubling_time']
    
    data=pd.DataFrame(deps_df.values,columns=headers_mat)
    print(data.head())

    data.to_csv("msc_upload.csv", index=False) #name your csv file the way you want and place the path
    
    conn_por=connect_neo4j_prod()
    # session_por=conn_por.session(database='mscdb')
    session_por=conn_por.session(database='neo4j')
    warnings.filterwarnings('ignore')

    drug_nodes1 = """
	    CALL apoc.periodic.iterate(
        "UNWIND $parameters as row RETURN row",
	    "MERGE (n:Drug{id:row.lib1_ID})
	    SET
	    n.maxc = row.lib1_maxc,
	    n.minc = row.lib1_minc,
        n.name = row.lib1_name,
        n.tag = row.lib1_tag;",
	    {batchSize:20000, parallel: false, params:{parameters: $parameters}});
	    """
    
    
    drug_nodes2 = """
	    CALL apoc.periodic.iterate(
        "UNWIND $parameters as row RETURN row",
	    "MERGE (n:Drug{id:row.lib2_ID})
	    SET
	    n.maxc = row.lib2_maxc,
	    n.minc = row.lib2_minc,
        n.name = row.lib2_name,
        n.tag = row.lib2_tag;",
	    {batchSize:20000, parallel: false, params:{parameters: $parameters}});
	    """

    query_list = [drug_nodes1,drug_nodes2]

    df_split = chunk_dataframe(data, 20000)
    for idx, df_each in enumerate(df_split):
        print ('load_to_neo4j: idx=', idx, 'df.len=', len(df_each))
        for each_query in query_list:
            parameters = {"parameters": df_each.to_dict(orient='records')}
            result =run_on_neo4j(each_query, parameters,session_por)
        print(result)
    

    conn_por.close()

In [ ]:
def genes():

    headers_mat=["comb_id","CELL_LINE_NAME","MASTER_CELL_ID","COSMIC_ID","TISSUE", "CANCER_TYPE","lib1_tag","lib1_ID","lib1_name","lib1_minc",'lib1_maxc','lib1_RMSE','lib1_MaxE',
                 'lib1_IC50_ln','lib1_target','lib1_pathway','lib2_tag','lib2_ID','lib2_name','lib2_minc','lib2_maxc','lib2_RMSE','lib2_MaxE','lib2_IC50_ln','lib2_target',
                 'lib2_pathway','combo_MaxE','Delta_MaxE_lib1','Delta_MaxE_lib2','Delta_combo_MaxE_day1','Bliss_synergistic_wells','Bliss_matrix','Bliss_matrix_SO',
                 'window_size','Bliss_window','Bliss_window_dose1','Bliss_window_dose2','Bliss_window_SO','Bliss_window_SO_dose1','Bliss_window_SO_dose2',
                 'HSA_synergistic_wells','HSA_matrix','HSA_matrix_SO','HSA_window','HSA_window_dose1','HSA_window_dose2','HSA_window_SO','HSA_window_SO_dose1',
                 'HSA_window_SO_dose2','day1_intensity_mean','day1_viability_mean','day1_inhibition_scale','growth_rate','doubling_time']
    
    data=pd.DataFrame(deps_df.values,columns=headers_mat)
    print(data.head())

    data.to_csv("msc_upload.csv", index=False) #name your csv file the way you want and place the path
    
    conn_por=connect_neo4j_prod()
    session_por=conn_por.session(database='mscdb')
    warnings.filterwarnings('ignore')

    target_nodes1 = """
	    CALL apoc.periodic.iterate(
        "UNWIND $parameters as row RETURN row",
	    "MERGE (n:Gene{name:row.lib1_target});",
	    {batchSize:20000, parallel: false, params:{parameters: $parameters}});
	    """
    
    target_nodes2 = """
	    CALL apoc.periodic.iterate(
        "UNWIND $parameters as row RETURN row",
	    "MERGE (n:Gene{name:row.lib2_target});",
	    {batchSize:20000, parallel: false, params:{parameters: $parameters}});
	    """

    
    query_list = [target_nodes1,target_nodes2]

    df_split = chunk_dataframe(data, 20000)
    for idx, df_each in enumerate(df_split):
        print ('load_to_neo4j: idx=', idx, 'df.len=', len(df_each))
        for each_query in query_list:
            parameters = {"parameters": df_each.to_dict(orient='records')}
            result =run_on_neo4j(each_query, parameters,session_por)
        print(result)
    

    conn_por.close()

In [ ]:
def pathways():

    headers_mat=["comb_id","CELL_LINE_NAME","MASTER_CELL_ID","COSMIC_ID","TISSUE", "CANCER_TYPE","lib1_tag","lib1_ID","lib1_name","lib1_minc",'lib1_maxc','lib1_RMSE','lib1_MaxE',
                 'lib1_IC50_ln','lib1_target','lib1_pathway','lib2_tag','lib2_ID','lib2_name','lib2_minc','lib2_maxc','lib2_RMSE','lib2_MaxE','lib2_IC50_ln','lib2_target',
                 'lib2_pathway','combo_MaxE','Delta_MaxE_lib1','Delta_MaxE_lib2','Delta_combo_MaxE_day1','Bliss_synergistic_wells','Bliss_matrix','Bliss_matrix_SO',
                 'window_size','Bliss_window','Bliss_window_dose1','Bliss_window_dose2','Bliss_window_SO','Bliss_window_SO_dose1','Bliss_window_SO_dose2',
                 'HSA_synergistic_wells','HSA_matrix','HSA_matrix_SO','HSA_window','HSA_window_dose1','HSA_window_dose2','HSA_window_SO','HSA_window_SO_dose1',
                 'HSA_window_SO_dose2','day1_intensity_mean','day1_viability_mean','day1_inhibition_scale','growth_rate','doubling_time']
    
    data=pd.DataFrame(deps_df.values,columns=headers_mat)
    print(data.head())

    data.to_csv("msc_upload.csv", index=False) #name your csv file the way you want and place the path
    
    conn_por=connect_neo4j_prod()
    session_por=conn_por.session(database='mscdb')
    warnings.filterwarnings('ignore')

    pathway_nodes1 = """
	    CALL apoc.periodic.iterate(
        "UNWIND $parameters as row RETURN row",
	    "MERGE (n:Pathway{name:row.lib1_pathway});",
	    {batchSize:20000, parallel: false, params:{parameters: $parameters}});
	    """
    
    pathway_nodes2 = """
	    CALL apoc.periodic.iterate(
        "UNWIND $parameters as row RETURN row",
	    "MERGE (n:Pathway{name:row.lib2_pathway});",
	    {batchSize:20000, parallel: false, params:{parameters: $parameters}});
	    """

    
    query_list = [pathway_nodes1,pathway_nodes2]

    df_split = chunk_dataframe(data, 20000)
    for idx, df_each in enumerate(df_split):
        print ('load_to_neo4j: idx=', idx, 'df.len=', len(df_each))
        for each_query in query_list:
            parameters = {"parameters": df_each.to_dict(orient='records')}
            result =run_on_neo4j(each_query, parameters,session_por)
        print(result)
    

    conn_por.close()

In [ ]:
def celllines():

    headers_mat=["comb_id","CELL_LINE_NAME","MASTER_CELL_ID","COSMIC_ID","TISSUE", "CANCER_TYPE","lib1_tag","lib1_ID","lib1_name","lib1_minc",'lib1_maxc','lib1_RMSE','lib1_MaxE',
                 'lib1_IC50_ln','lib1_target','lib1_pathway','lib2_tag','lib2_ID','lib2_name','lib2_minc','lib2_maxc','lib2_RMSE','lib2_MaxE','lib2_IC50_ln','lib2_target',
                 'lib2_pathway','combo_MaxE','Delta_MaxE_lib1','Delta_MaxE_lib2','Delta_combo_MaxE_day1','Bliss_synergistic_wells','Bliss_matrix','Bliss_matrix_SO',
                 'window_size','Bliss_window','Bliss_window_dose1','Bliss_window_dose2','Bliss_window_SO','Bliss_window_SO_dose1','Bliss_window_SO_dose2',
                 'HSA_synergistic_wells','HSA_matrix','HSA_matrix_SO','HSA_window','HSA_window_dose1','HSA_window_dose2','HSA_window_SO','HSA_window_SO_dose1',
                 'HSA_window_SO_dose2','day1_intensity_mean','day1_viability_mean','day1_inhibition_scale','growth_rate','doubling_time']
    
    data=pd.DataFrame(deps_df.values,columns=headers_mat)
    print(data.head())

    data.to_csv("msc_upload.csv", index=False) #name your csv file the way you want and place the path
    
    conn_por=connect_neo4j_prod()
    session_por=conn_por.session(database='mscdb')
    warnings.filterwarnings('ignore')

    cellLine_nodes = """
	    CALL apoc.periodic.iterate(
        "UNWIND $parameters as row RETURN row",
	    "MERGE (n:CellLine{id:row.MASTER_CELL_ID})
        SET
	    n.cancer_type = row.CANCER_TYPE,
	    n.tissue = row.TISSUE,
        n.name = row.CELL_LINE_NAME;",
	    {batchSize:20000, parallel: false, params:{parameters: $parameters}});
	    """

    
    query_list = [cellLine_nodes]

    df_split = chunk_dataframe(data, 20000)
    for idx, df_each in enumerate(df_split):
        print ('load_to_neo4j: idx=', idx, 'df.len=', len(df_each))
        for each_query in query_list:
            parameters = {"parameters": df_each.to_dict(orient='records')}
            result =run_on_neo4j(each_query, parameters,session_por)
        print(result)
    

    conn_por.close()

In [ ]:
def combinations():

    headers_mat=["comb_id","CELL_LINE_NAME","MASTER_CELL_ID","COSMIC_ID","TISSUE", "CANCER_TYPE","lib1_tag","lib1_ID","lib1_name","lib1_minc",'lib1_maxc',
                 'lib1_RMSE','lib1_MaxE',
                 'lib1_IC50_ln','lib1_target','lib1_pathway','lib2_tag','lib2_ID','lib2_name','lib2_minc','lib2_maxc','lib2_RMSE','lib2_MaxE',
                 'lib2_IC50_ln','lib2_target',
                 'lib2_pathway','combo_MaxE','Delta_MaxE_lib1','Delta_MaxE_lib2','Delta_combo_MaxE_day1','Bliss_synergistic_wells','Bliss_matrix',
                 'Bliss_matrix_SO',
                 'window_size','Bliss_window','Bliss_window_dose1','Bliss_window_dose2','Bliss_window_SO','Bliss_window_SO_dose1','Bliss_window_SO_dose2',
                 'HSA_synergistic_wells','HSA_matrix','HSA_matrix_SO','HSA_window','HSA_window_dose1','HSA_window_dose2','HSA_window_SO',
                 'HSA_window_SO_dose1',
                 'HSA_window_SO_dose2','day1_intensity_mean','day1_viability_mean','day1_inhibition_scale','growth_rate','doubling_time']
    
    data=pd.DataFrame(deps_df.values,columns=headers_mat)
    print(data.head())

    data.to_csv("msc_upload.csv", index=False) #name your csv file the way you want and place the path
    
    conn_por=connect_neo4j_prod()
    session_por=conn_por.session(database='mscdb')
    warnings.filterwarnings('ignore')

    combination_nodes = """
	    CALL apoc.periodic.iterate(
        "UNWIND $parameters as row RETURN row",
	    "MERGE (n:Combination{id:row.comb_id})
        SET
	    n.bliss_matrix = row.Bliss_matrix,
	    n.bliss_matrix_so = row.Bliss_matrix_SO,
        n.bliss_synergistic_wells = row.Bliss_synergistic_wells,
        n.bliss_window = row.Bliss_window ,
        n.bliss_window_dose1 = row.Bliss_window_dose1 ,
        n.bliss_window_dose2 = row.Bliss_window_dose2 ,
        n.bliss_window_so = row.Bliss_window_SO ,
        n.bliss_window_so_dose1 = row.Bliss_window_SO_dose1 ,
        n.bliss_window_so_dose2 = row.Bliss_window_SO_dose2 ,
        n.combo_maxe = row.combo_MaxE ,
        n.day1_inhibition_scale = row.day1_inhibition_scale ,
        n.day1_intensity_mean = row.day1_intensity_mean ,
        n.day1_viability_mean = row.day1_viability_mean ,
        n.delta_combo_maxe_day1 = row.Delta_combo_MaxE_day1 ,
        n.delta_maxe_lib1 = row.Delta_MaxE_lib1 ,
        n.delta_maxe_lib2 = row.Delta_MaxE_lib2 ,
        n.doubling_time = row.doubling_time ,
        n.growth_rate = row.growth_rate ,
        n.hsa_matrix = row.HSA_matrix ,
        n.hsa_matrix_so = row.HSA_matrix_SO ,
        n.hsa_synergistic_wells = row.HSA_synergistic_wells ,
        n.hsa_window = row.HSA_window ,
        n.hsa_window_dose1 = row.HSA_window_dose1 ,
        n.hsa_window_dose2 = row.HSA_window_dose2 ,
        n.hsa_window_so= row.HSA_window_SO ,
        n.hsa_window_so_dose1 = row.HSA_window_SO_dose1 ,
        n.hsa_window_so_dose2 = row.HSA_window_SO_dose2 ,
        n.window_size= row.window_size,
        n.drug1_name= row.lib1_name,
        n.drug2_name= row.lib2_name,
        n.drug1_rmse= row.lib1_RMSE,
        n.drug2_rmse= row.lib2_RMSE,
        n.drug1_maxe= row.lib1_MaxE,
        n.drug2_maxe= row.lib2_MaxE,
        n.drug1_ic50_ln= row.lib1_IC50_ln,
        n.drug2_ic50_ln= row.lib2_IC50_ln,
        n.data_source='007';",
	    {batchSize:20000, parallel: false, params:{parameters: $parameters}});
	    """

    
    query_list = [combination_nodes]

    df_split = chunk_dataframe(data, 20000)
    for idx, df_each in enumerate(df_split):
        print ('load_to_neo4j: idx=', idx, 'df.len=', len(df_each))
        for each_query in query_list:
            parameters = {"parameters": df_each.to_dict(orient='records')}
            result =run_on_neo4j(each_query, parameters,session_por)
        print(result)
    

    conn_por.close()

In [ ]:
def contains_rels():

    headers_mat=["comb_id","CELL_LINE_NAME","MASTER_CELL_ID","COSMIC_ID","TISSUE", "CANCER_TYPE","lib1_tag","lib1_ID","lib1_name","lib1_minc",'lib1_maxc',
                 'lib1_RMSE','lib1_MaxE',
                 'lib1_IC50_ln','lib1_target','lib1_pathway','lib2_tag','lib2_ID','lib2_name','lib2_minc','lib2_maxc','lib2_RMSE','lib2_MaxE',
                 'lib2_IC50_ln','lib2_target',
                 'lib2_pathway','combo_MaxE','Delta_MaxE_lib1','Delta_MaxE_lib2','Delta_combo_MaxE_day1','Bliss_synergistic_wells','Bliss_matrix',
                 'Bliss_matrix_SO',
                 'window_size','Bliss_window','Bliss_window_dose1','Bliss_window_dose2','Bliss_window_SO','Bliss_window_SO_dose1','Bliss_window_SO_dose2',
                 'HSA_synergistic_wells','HSA_matrix','HSA_matrix_SO','HSA_window','HSA_window_dose1','HSA_window_dose2','HSA_window_SO',
                 'HSA_window_SO_dose1',
                 'HSA_window_SO_dose2','day1_intensity_mean','day1_viability_mean','day1_inhibition_scale','growth_rate','doubling_time']
    
    data=pd.DataFrame(deps_df.values,columns=headers_mat)
    print(data.head())

    data.to_csv("msc_upload.csv", index=False) #name your csv file the way you want and place the path
    
    conn_por=connect_neo4j_prod()
    session_por=conn_por.session(database='mscdb')
    warnings.filterwarnings('ignore')


    contains1_rel ="""
        CALL apoc.periodic.iterate(
        "UNWIND $parameters as row RETURN row",
        "MATCH (m:Combination {id:row.comb_id})
        MATCH (r:Drug{id:row.lib1_ID})
        MERGE (m)-[f:CONTAINS]->(r)
        ;",
        {batchSize:2000, parallel: false, params:{parameters: $parameters}});
    """
    contains2_rel ="""
        CALL apoc.periodic.iterate(
        "UNWIND $parameters as row RETURN row",
        "MATCH (m:Combination {id:row.comb_id})
        MATCH (r:Drug{id:row.lib2_ID})
        MERGE (m)-[f:CONTAINS]->(r)
        ;",
        {batchSize:2000, parallel: false, params:{parameters: $parameters}});
    """
    query_list = [contains1_rel,contains2_rel]

    df_split = chunk_dataframe(data, 20000)
    for idx, df_each in enumerate(df_split):
        print ('load_to_neo4j: idx=', idx, 'df.len=', len(df_each))
        for each_query in query_list:
            parameters = {"parameters": df_each.to_dict(orient='records')}
            result =run_on_neo4j(each_query, parameters,session_por)
        print(result)
    

    conn_por.close()

In [ ]:
def tested_rels():

    headers_mat=["comb_id","CELL_LINE_NAME","MASTER_CELL_ID","COSMIC_ID","TISSUE", "CANCER_TYPE","lib1_tag","lib1_ID","lib1_name","lib1_minc",'lib1_maxc',
                 'lib1_RMSE','lib1_MaxE',
                 'lib1_IC50_ln','lib1_target','lib1_pathway','lib2_tag','lib2_ID','lib2_name','lib2_minc','lib2_maxc','lib2_RMSE','lib2_MaxE',
                 'lib2_IC50_ln','lib2_target',
                 'lib2_pathway','combo_MaxE','Delta_MaxE_lib1','Delta_MaxE_lib2','Delta_combo_MaxE_day1','Bliss_synergistic_wells','Bliss_matrix',
                 'Bliss_matrix_SO',
                 'window_size','Bliss_window','Bliss_window_dose1','Bliss_window_dose2','Bliss_window_SO','Bliss_window_SO_dose1','Bliss_window_SO_dose2',
                 'HSA_synergistic_wells','HSA_matrix','HSA_matrix_SO','HSA_window','HSA_window_dose1','HSA_window_dose2','HSA_window_SO',
                 'HSA_window_SO_dose1',
                 'HSA_window_SO_dose2','day1_intensity_mean','day1_viability_mean','day1_inhibition_scale','growth_rate','doubling_time']
    
    data=pd.DataFrame(deps_df.values,columns=headers_mat)
    print(data.head())

    data.to_csv("msc_upload.csv", index=False) #name your csv file the way you want and place the path
    
    conn_por=connect_neo4j_prod()
    session_por=conn_por.session(database='mscdb')
    warnings.filterwarnings('ignore')


    tested1_rel ="""
        CALL apoc.periodic.iterate(
        "UNWIND $parameters as row RETURN row",
        "MATCH (m:Combination {id:row.comb_id})
        MATCH (r:CellLine{id:row.MASTER_CELL_ID})
        MERGE (m)-[f:TESTED_ON]->(r)
        ;",
        {batchSize:2000, parallel: false, params:{parameters: $parameters}});
    """
    tested2_rel ="""
        CALL apoc.periodic.iterate(
        "UNWIND $parameters as row RETURN row",
        "MATCH (m:Drug{id:row.lib1_ID})
        MATCH (r:CellLine{id:row.MASTER_CELL_ID})
        MERGE (m)-[f:TESTED_ON]->(r)
        ;",
        {batchSize:2000, parallel: false, params:{parameters: $parameters}});
    """

    tested3_rel ="""
        CALL apoc.periodic.iterate(
        "UNWIND $parameters as row RETURN row",
        "MATCH (m:Drug{id:row.lib2_ID})
        MATCH (r:CellLine{id:row.MASTER_CELL_ID})
        MERGE (m)-[f:TESTED_ON]->(r)
        ;",
        {batchSize:2000, parallel: false, params:{parameters: $parameters}});
    """

    query_list = [tested1_rel,tested2_rel,tested3_rel]

    df_split = chunk_dataframe(data, 20000)
    for idx, df_each in enumerate(df_split):
        print ('load_to_neo4j: idx=', idx, 'df.len=', len(df_each))
        for each_query in query_list:
            parameters = {"parameters": df_each.to_dict(orient='records')}
            result =run_on_neo4j(each_query, parameters,session_por)
        print(result)
    

    conn_por.close()

In [ ]:
def targets_rels():

    headers_mat=["comb_id","CELL_LINE_NAME","MASTER_CELL_ID","COSMIC_ID","TISSUE", "CANCER_TYPE","lib1_tag","lib1_ID","lib1_name","lib1_minc",'lib1_maxc',
                 'lib1_RMSE','lib1_MaxE',
                 'lib1_IC50_ln','lib1_target','lib1_pathway','lib2_tag','lib2_ID','lib2_name','lib2_minc','lib2_maxc','lib2_RMSE','lib2_MaxE',
                 'lib2_IC50_ln','lib2_target',
                 'lib2_pathway','combo_MaxE','Delta_MaxE_lib1','Delta_MaxE_lib2','Delta_combo_MaxE_day1','Bliss_synergistic_wells','Bliss_matrix',
                 'Bliss_matrix_SO',
                 'window_size','Bliss_window','Bliss_window_dose1','Bliss_window_dose2','Bliss_window_SO','Bliss_window_SO_dose1','Bliss_window_SO_dose2',
                 'HSA_synergistic_wells','HSA_matrix','HSA_matrix_SO','HSA_window','HSA_window_dose1','HSA_window_dose2','HSA_window_SO',
                 'HSA_window_SO_dose1',
                 'HSA_window_SO_dose2','day1_intensity_mean','day1_viability_mean','day1_inhibition_scale','growth_rate','doubling_time']
    
    data=pd.DataFrame(deps_df.values,columns=headers_mat)
    print(data.head())

    data.to_csv("msc_upload.csv", index=False) #name your csv file the way you want and place the path
    
    conn_por=connect_neo4j_prod()
    session_por=conn_por.session(database='mscdb')
    warnings.filterwarnings('ignore')


    targets1_rel ="""
        CALL apoc.periodic.iterate(
        "UNWIND $parameters as row RETURN row",
        "MATCH (m:Drug{id:row.lib1_ID})
        MATCH (r:Pathway{name:row.lib1_pathway})
        MERGE (m)-[f:TARGETS]->(r)
        ;",
        {batchSize:2000, parallel: false, params:{parameters: $parameters}});
    """
    targets2_rel ="""
        CALL apoc.periodic.iterate(
        "UNWIND $parameters as row RETURN row",
        "MATCH (m:Drug{id:row.lib2_ID})
        MATCH (r:Pathway{name:row.lib2_pathway})
        MERGE (m)-[f:TARGETS]->(r)
        ;",
        {batchSize:2000, parallel: false, params:{parameters: $parameters}});
    """
    targets3_rel ="""
        CALL apoc.periodic.iterate(
        "UNWIND $parameters as row RETURN row",
        "MATCH (m:Drug{id:row.lib1_ID})
        MATCH (r:Gene{name:row.lib1_target})
        MERGE (m)-[f:TARGETS]->(r)
        ;",
        {batchSize:2000, parallel: false, params:{parameters: $parameters}});
    """
    targets4_rel ="""
        CALL apoc.periodic.iterate(
        "UNWIND $parameters as row RETURN row",
        "MATCH (m:Drug{id:row.lib2_ID})
        MATCH (r:Gene{name:row.lib2_target})
        MERGE (m)-[f:TARGETS]->(r)
        ;",
        {batchSize:2000, parallel: false, params:{parameters: $parameters}});
    """

    query_list = [targets1_rel,targets2_rel,targets3_rel,targets4_rel]

    df_split = chunk_dataframe(data, 20000)
    for idx, df_each in enumerate(df_split):
        print ('load_to_neo4j: idx=', idx, 'df.len=', len(df_each))
        for each_query in query_list:
            parameters = {"parameters": df_each.to_dict(orient='records')}
            result =run_on_neo4j(each_query, parameters,session_por)
        print(result)
    

    conn_por.close()

In [ ]:
def within_rels():

    headers_mat=["comb_id","CELL_LINE_NAME","MASTER_CELL_ID","COSMIC_ID","TISSUE", "CANCER_TYPE","lib1_tag","lib1_ID","lib1_name","lib1_minc",'lib1_maxc',
                 'lib1_RMSE','lib1_MaxE',
                 'lib1_IC50_ln','lib1_target','lib1_pathway','lib2_tag','lib2_ID','lib2_name','lib2_minc','lib2_maxc','lib2_RMSE','lib2_MaxE',
                 'lib2_IC50_ln','lib2_target',
                 'lib2_pathway','combo_MaxE','Delta_MaxE_lib1','Delta_MaxE_lib2','Delta_combo_MaxE_day1','Bliss_synergistic_wells','Bliss_matrix',
                 'Bliss_matrix_SO',
                 'window_size','Bliss_window','Bliss_window_dose1','Bliss_window_dose2','Bliss_window_SO','Bliss_window_SO_dose1','Bliss_window_SO_dose2',
                 'HSA_synergistic_wells','HSA_matrix','HSA_matrix_SO','HSA_window','HSA_window_dose1','HSA_window_dose2','HSA_window_SO',
                 'HSA_window_SO_dose1',
                 'HSA_window_SO_dose2','day1_intensity_mean','day1_viability_mean','day1_inhibition_scale','growth_rate','doubling_time']
    
    data=pd.DataFrame(deps_df.values,columns=headers_mat)
    print(data.head())

    data.to_csv("msc_upload.csv", index=False) #name your csv file the way you want and place the path
    
    conn_por=connect_neo4j_prod()
    session_por=conn_por.session(database='mscdb')
    warnings.filterwarnings('ignore')


    within1_rel ="""
        CALL apoc.periodic.iterate(
        "UNWIND $parameters as row RETURN row",
        "MATCH (m:Gene{name:row.lib1_target})
        MATCH (r:Pathway{name:row.lib1_pathway})
        MERGE (m)-[f:WITHIN]->(r)
        ;",
        {batchSize:2000, parallel: false, params:{parameters: $parameters}});
    """
    within2_rel ="""
        CALL apoc.periodic.iterate(
        "UNWIND $parameters as row RETURN row",
        "MATCH (m:Gene{name:row.lib2_target})
        MATCH (r:Pathway{name:row.lib2_pathway})
        MERGE (m)-[f:WITHIN]->(r)
        ;",
        {batchSize:2000, parallel: false, params:{parameters: $parameters}});
    """


    query_list = [within1_rel,within2_rel]

    df_split = chunk_dataframe(data, 20000)
    for idx, df_each in enumerate(df_split):
        print ('load_to_neo4j: idx=', idx, 'df.len=', len(df_each))
        for each_query in query_list:
            parameters = {"parameters": df_each.to_dict(orient='records')}
            result =run_on_neo4j(each_query, parameters,session_por)
        print(result)
    

    conn_por.close()

In [ ]:
def expressed_rels():

    headers_mat=["comb_id","CELL_LINE_NAME","MASTER_CELL_ID","COSMIC_ID","TISSUE", "CANCER_TYPE","lib1_tag","lib1_ID","lib1_name","lib1_minc",'lib1_maxc',
                 'lib1_RMSE','lib1_MaxE',
                 'lib1_IC50_ln','lib1_target','lib1_pathway','lib2_tag','lib2_ID','lib2_name','lib2_minc','lib2_maxc','lib2_RMSE','lib2_MaxE',
                 'lib2_IC50_ln','lib2_target',
                 'lib2_pathway','combo_MaxE','Delta_MaxE_lib1','Delta_MaxE_lib2','Delta_combo_MaxE_day1','Bliss_synergistic_wells','Bliss_matrix',
                 'Bliss_matrix_SO',
                 'window_size','Bliss_window','Bliss_window_dose1','Bliss_window_dose2','Bliss_window_SO','Bliss_window_SO_dose1','Bliss_window_SO_dose2',
                 'HSA_synergistic_wells','HSA_matrix','HSA_matrix_SO','HSA_window','HSA_window_dose1','HSA_window_dose2','HSA_window_SO',
                 'HSA_window_SO_dose1',
                 'HSA_window_SO_dose2','day1_intensity_mean','day1_viability_mean','day1_inhibition_scale','growth_rate','doubling_time']
    
    data=pd.DataFrame(deps_df.values,columns=headers_mat)
    print(data.head())

    data.to_csv("msc_upload.csv", index=False) #name your csv file the way you want and place the path
    
    conn_por=connect_neo4j_prod()
    session_por=conn_por.session(database='mscdb')
    warnings.filterwarnings('ignore')


    expressed1_rel ="""
        CALL apoc.periodic.iterate(
        "UNWIND $parameters as row RETURN row",
        "MATCH (m:Gene{name:row.lib1_target})
        MATCH (r:CellLine{id:row.MASTER_CELL_ID})
        MERGE (m)-[f:EXPRESSED_IN]->(r)
        ;",
        {batchSize:2000, parallel: false, params:{parameters: $parameters}});
    """
    expressed2_rel  ="""
        CALL apoc.periodic.iterate(
        "UNWIND $parameters as row RETURN row",
        "MATCH (m:Gene{name:row.lib2_target})
        MATCH (r:CellLine{id:row.MASTER_CELL_ID})
        MERGE (m)-[f:EXPRESSED_IN]->(r)
        ;",
        {batchSize:2000, parallel: false, params:{parameters: $parameters}});
    """


    query_list = [expressed1_rel,expressed2_rel]

    df_split = chunk_dataframe(data, 20000)
    for idx, df_each in enumerate(df_split):
        print ('load_to_neo4j: idx=', idx, 'df.len=', len(df_each))
        for each_query in query_list:
            parameters = {"parameters": df_each.to_dict(orient='records')}
            result =run_on_neo4j(each_query, parameters,session_por)
        print(result)
    

    conn_por.close()

In [22]:
def main():

    drugs()
    #genes()
    #pathways()
    #celllines()
    #combinations()
    #contains_rels()
    #tested_rels()
    #targets_rels()
    #within_rels()
    #expressed_rels()
    
main()

  comb_id CELL_LINE_NAME MASTER_CELL_ID COSMIC_ID TISSUE CANCER_TYPE lib1_tag  \
0       1          A2058            652    906792   Skin    Melanoma      L56   
1       2          A2058            652    906792   Skin    Melanoma      L57   
2       3          A2058            652    906792   Skin    Melanoma      L58   
3       4          A2058            652    906792   Skin    Melanoma      L59   
4       5          A2058            652    906792   Skin    Melanoma      L60   

  lib1_ID lib1_name lib1_minc  ... HSA_window_dose1 HSA_window_dose2  \
0    2435   AZD6738      0.01  ...               D5               D5   
1    2436   AZD1775     0.001  ...               D5               D5   
2    2436   AZD1775     0.001  ...               D3               D1   
3    2428   AZD2811     0.001  ...               D3               D5   
4    2428   AZD2811     0.001  ...               D5               D3   

  HSA_window_SO HSA_window_SO_dose1 HSA_window_SO_dose2 day1_intensity_mean  \
0

Failed to read from defunct connection IPv4Address(('48ee84aa.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.69.128.95', 7687)))
Traceback (most recent call last):
  File "c:\Users\chrisk41\AppData\Local\Programs\Python\Python310\lib\site-packages\neo4j\_sync\io\_common.py", line 53, in _buffer_one_chunk
    receive_into_buffer(self._socket, self._buffer, 2)
  File "c:\Users\chrisk41\AppData\Local\Programs\Python\Python310\lib\site-packages\neo4j\_sync\io\_common.py", line 310, in receive_into_buffer
    raise OSError("No data")
OSError: No data

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\chrisk41\AppData\Local\Temp\4\ipykernel_26592\69599056.py", line 32, in run_on_neo4j
    result = [record.data() for record in curs]
  File "C:\Users\chrisk41\AppData\Local\Temp\4\ipykernel_26592\69599056.py", line 32, in <listcomp>
    result = [record.data() for record in curs]
  File "c:\Users\chrisk41\AppData\Loc

Error during Cypher query execution: Failed to read from defunct connection IPv4Address(('48ee84aa.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.69.128.95', 7687)))


Failed to read from defunct connection ResolvedIPv4Address(('34.69.128.95', 7687)) (ResolvedIPv4Address(('34.69.128.95', 7687)))
Unable to retrieve routing information


Error during Cypher query execution: Unable to retrieve routing information
error


Traceback (most recent call last):
  File "C:\Users\chrisk41\AppData\Local\Temp\4\ipykernel_26592\69599056.py", line 30, in run_on_neo4j
    tx = session_por.begin_transaction()
  File "c:\Users\chrisk41\AppData\Local\Programs\Python\Python310\lib\site-packages\neo4j\_sync\work\session.py", line 475, in begin_transaction
    self._open_transaction(
  File "c:\Users\chrisk41\AppData\Local\Programs\Python\Python310\lib\site-packages\neo4j\_sync\work\session.py", line 411, in _open_transaction
    self._connect(access_mode=access_mode)
  File "c:\Users\chrisk41\AppData\Local\Programs\Python\Python310\lib\site-packages\neo4j\_sync\work\session.py", line 135, in _connect
    super()._connect(
  File "c:\Users\chrisk41\AppData\Local\Programs\Python\Python310\lib\site-packages\neo4j\_sync\work\workspace.py", line 178, in _connect
    self._connection = self._pool.acquire(**acquire_kwargs_)
  File "c:\Users\chrisk41\AppData\Local\Programs\Python\Python310\lib\site-packages\neo4j\_sync\io\_pool